In [ ]:
import urllib3
import json
from urllib.parse import urlparse, parse_qs
import requests
import urllib
import pandas as pd
from io import BytesIO

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# id1,id2 july
# ph1, ph2, ph3
region = {
    'id1': {
        'channel_id': 1753200,
        'timezone': 'Asia/Kolkata',
        'key': '6ZW482KDBT7BEC47'
    },
    'id2': {
        'channel_id': 1754117,
        'timezone': 'Asia/Kolkata',
        'key': 'VZMWN89XGM66OZOP'
    },
    'id3': {
        'channel_id': 1754130,
        'timezone': 'Asia/Kolkata',
        'key': 'NYHTIPSAYT1D3H0B'
    },
    'ph1': {
        'channel_id': 1754137,
        'timezone': 'Asia/Kuala_Lumpur',
        'key': '3UTNSLQM6AZ71T5O'
    },
    'ph2': {
        'channel_id': 1754143,
        'timezone': 'Asia/Kuala_Lumpur',
        'key': '09BKCR4057NJD380'
    },
    'ph3': {
        'channel_id': 1754148,
        'timezone': 'Asia/Kuala_Lumpur',
        'key': 'XTTZ8OHDLZBQ4PHP'
    },
    'my1': {
        'channel_id': 1771970,
        'timezone': 'Asia/Kuala_Lumpur',
        'key': 'EI8VXDPLEMDPZWYJ'
    },
    'my2': {
        'channel_id': 1771974,
        'timezone': 'Asia/Kuala_Lumpur',
        'key': 'ZGEH65SH587P9Y0Z'
    },
    'my3': {
        'channel_id': 1771976,
        'timezone': 'Asia/Kuala_Lumpur',
        'key': 'WZZTUUBU8WU9H9JT'
    }
}

days_in_a_month = {
    'january': {'index': 1, 'start': 1, 'end': 31, 'status': False, 'year': 2023},
    'february': {'index': 2, 'start': 1, 'end': 28, 'status': False, 'year': 2023},
    'march': {'index': 3, 'start': 1, 'end': 31, 'status': False, 'year': 2023},
    'april': {'index': 4, 'start': 1, 'end': 21, 'status': False, 'year': 2023},
    'may': {'index': 5, 'start': 1, 'end': 1, 'status': True, 'year': 2023},
    'june': {'index': 6, 'start': 1, 'end': 30, 'status': False, 'year': 2023},
    'july': {'index': 7, 'start': 1, 'end': 31, 'status': False, 'year': 2022},
    'august': {'index': 8, 'start': 1, 'end': 31, 'status': False, 'year': 2022},
    'september': {'index': 9, 'start': 1, 'end': 30, 'status': False, 'year': 2022},
    'october': {'index': 10, 'start': 1, 'end': 31, 'status': False, 'year': 2022},
    'november': {'index': 11, 'start': 1, 'end': 30, 'status': False, 'year': 2022},
    'december': {'index': 12, 'start': 1, 'end': 31, 'status': False, 'year': 2022},
}

alias = 'my3'

http = urllib3.PoolManager()
API_KEY = region[alias]['key']
timezone = region[alias]['timezone']
channel_id = region[alias]['channel_id']
filename = alias+'_'+str(channel_id)+'.csv'

# "https://api.thingspeak.com/channels/1753200/feeds.csv?start=2023-03-02%E2%80%99%E2%80%9900:00:00%E2%80%99%E2%80%99&end=2023-03-02%E2%80%99%E2%80%9924:00:00%E2%80%99%E2%80%99&api_key=NYHTIPSAYT1D3H0B&timezone=Asia/Kolkata"
URL = "https://api.thingspeak.com/channels/1753200/feeds.csv?start=2023-03-02%E2%80%99%E2%80%9900:00:00%E2%80%99%E2" \
      "%80%99&end=2023-03-02%E2%80%99%E2%80%9924:00:00%E2%80%99%E2%80%99&api_key=NYHTIPSAYT1D3H0B&timezone=Asia" \
      "/Kolkata "
URL_main = 'https://api.thingspeak.com/'
URL_path = f'channels/{channel_id}/feeds.csv'
URL_params = f'?start=2023-03-03%E2%80%99%E2%80%9900:00:00%E2%80%99%E2%80%99&end=2023-03-03%E2%80%99%E2%80%9924:00:00' \
             f'%E2%80%99%E2%80%99&api_key={API_KEY}&timezone={timezone} '

netUrl = URL_main + URL_path + URL_params

parsed_url = urlparse(URL)
queries = parse_qs(parsed_url.query)




In [ ]:
def convert_date(raw_query, day, month, year):
    arr = raw_query.split("-")
    arr[0] = str(year)
    arr[1] = str(month)
    arr[2] = arr[2].replace(arr[2][:2], day)
    new_temp = arr[0] + '-' + arr[1] + '-' + arr[2]
    return new_temp


def convert_url(url, dayStart, dayEnd, api_key, timezone):
    payload = {
        'format': 'json',
        'start': dayStart,
        'end': dayEnd,
        'api_key': api_key,
        'timezone': timezone
    }
    payload_str = urllib.parse.urlencode(payload, safe=':')
    res = requests.get(
        url,
        params=payload_str
    )
    return res.content




In [ ]:
hasHeader = False
df = pd.DataFrame()
headerList = ['created_at', 'entry_id', 'field1', 'field2', 'field3', 'field4', 'field5', 'field6', 'field7', 'field8']

In [ ]:
print('please wait ....')
for i in days_in_a_month:
    data_in_csv = pd.DataFrame()

    if days_in_a_month[i]['status']:
        month = days_in_a_month[i]['index']
        year = days_in_a_month[i]['year']
        temp_url = netUrl
        for day in range(days_in_a_month[i]['end']):
            tempDay = str(day + 1).zfill(2)
            tempMonth = str(month).zfill(2)
            print('downloading ...')
            data = convert_url(URL_main + URL_path,
                               convert_date(queries['start'][0], tempDay, tempMonth, year),
                               convert_date(queries['end'][0], tempDay, tempMonth, year),
                               API_KEY,
                               timezone)
            if hasHeader:
                df = pd.read_csv(BytesIO(data))
                df.to_csv('/content/drive/MyDrive/TestData/'+filename, mode='a', header=None, index=None)
            else:
                df = pd.read_csv(BytesIO(data))
                df.to_csv('/content/drive/MyDrive/TestData/'+filename, mode='a', header=headerList, index=None)
                hasHeader = True

            print('data of', str(tempDay) + '-' + str(tempMonth) + '-' + str(year), 'added to file')

print('done with extracting data')


please wait ....
downloading ...
data of 01-05-2023 added to file
done with extracting data
